<a href="https://colab.research.google.com/github/anujb6/Swing-Filterer/blob/main/Swing_Trade_Stock_Picker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tradingview-ta

In [ ]:
import pandas as pd
import numpy as np
from tradingview_ta import TA_Handler, Interval, Exchange
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/UNIVERSE_SYMBOL.csv')
df.head()

,id,UNIVERSE,SYMBOL
0,1,NIFTY 50,ADANIENT
1,2,NIFTY 50,ADANIPORTS
2,3,NIFTY 50,APOLLOHOSP
3,4,NIFTY 50,ASIANPAINT
4,5,NIFTY 50,AXISBANK


In [ ]:
tickers_data = []
tickers = df['SYMBOL'].tolist()
removeTickers = []

for ticker in tickers:
    try:
        data = TA_Handler(
            symbol=ticker,
            screener="india",
            exchange="NSE",
            interval="1d"
        )
        data = data.get_analysis().summary
        tickers_data.append(data)

    except Exception as e:
        print(f"No data found for ticker {ticker} in NYSE. Searching in NASDAQ...")
        removeTickers.append(ticker)

for ticker in removeTickers:
  tickers.remove(ticker)

print("Data successfully imported.")

No data found for ticker MAHINDCIE in NYSE. Searching in NASDAQ...
No data found for ticker BAJAJ-AUTO in NYSE. Searching in NASDAQ...
No data found for ticker ARE&M in NYSE. Searching in NASDAQ...
No data found for ticker NAM-INDIA in NYSE. Searching in NASDAQ...
No data found for ticker MCDOWELL-N in NYSE. Searching in NASDAQ...
No data found for ticker M&MFIN in NYSE. Searching in NASDAQ...
No data found for ticker ARE&M in NYSE. Searching in NASDAQ...
No data found for ticker NAM-INDIA in NYSE. Searching in NASDAQ...
No data found for ticker BAJAJ-AUTO in NYSE. Searching in NASDAQ...
No data found for ticker M&M in NYSE. Searching in NASDAQ...
No data found for ticker M&MFIN in NYSE. Searching in NASDAQ...
No data found for ticker NIMI150 in NYSE. Searching in NASDAQ...
No data found for ticker BANKNIFTY24JANFUT in NYSE. Searching in NASDAQ...
No data found for ticker FINNIFTY24JANFUT in NYSE. Searching in NASDAQ...
No data found for ticker NIFTY24JANFUT in NYSE. Searching in NASDA

In [ ]:
recommendations = []
buys = []
sells = []
neutrals = []

for data in tickers_data:
    recommendation = data.get('RECOMMENDATION')
    buy = data.get('BUY')
    sell = data.get('SELL')
    neutral = data.get('NEUTRAL')

    recommendations.append(recommendation)
    buys.append(buy)
    sells.append(sell)
    neutrals.append(neutral)

data = {
    'Ticker': tickers,
    'Recommendations': recommendations,
    'Buys': buys,
    'Sells': sells,
    'Neutrals': neutrals
}

df = pd.DataFrame(data)
df

,Ticker,Recommendations,Buys,Sells,Neutrals
0,ADANIENT,SELL,4,13,9
1,ADANIPORTS,SELL,5,11,10
2,APOLLOHOSP,BUY,11,6,9
3,ASIANPAINT,SELL,1,15,10
4,AXISBANK,SELL,3,13,10
...,...,...,...,...,...
1178,OFSS,SELL,5,13,8
1179,PVRINOX,NEUTRAL,7,9,10
1180,RBLBANK,SELL,3,13,10
1181,NIFTY,SELL,5,11,10


In [ ]:
# Defining the order of categories
order_categories = {
    'STRONG_BUY': 5,
    'BUY': 4,
    'NEUTRAL': 3,
    'SELL': 2,
    'STRONG_SELL': 1
}

# Assigning a numerical value to each category in a new column "Order"
df['Order'] = df['Recommendations'].map(order_categories)
df = df.sort_values('Order', ascending=True).reset_index(drop=True)

# Droping the "Order" column if not needed in the final output
df = df.drop('Order', axis=1)

df

,Ticker,Recommendations,Buys,Sells,Neutrals
0,BERGEPAINT,STRONG_SELL,0,16,10
1,RBA,STRONG_SELL,1,16,9
2,COFORGE,STRONG_SELL,0,17,9
3,BANDHANBNK,STRONG_SELL,1,16,9
4,BANDHANBNK,STRONG_SELL,1,16,9
...,...,...,...,...,...
1178,MOTHERSON,STRONG_BUY,18,0,8
1179,RADICO,STRONG_BUY,16,1,9
1180,CGPOWER,STRONG_BUY,17,1,8
1181,SAREGAMA,STRONG_BUY,18,0,8


In [ ]:
df_adrs_indicators = pd.DataFrame()

for ticker in tickers:
    try:
        data = TA_Handler(
            symbol=ticker,
            screener="india",
            exchange="NSE",
            interval="1d"
        )
        data = data.get_analysis().indicators

    except Exception as e:
        print(f"No data found for ticker {ticker} in NYSE. Searching in NASDAQ...")
        data = TA_Handler(
            symbol=ticker,
            screener="india",
            exchange="NSE",
            interval="1d"
        )
        data = data.get_analysis().indicators

    df_temp = pd.DataFrame(data, index=[ticker])
    df_adrs_indicators = pd.concat([df_adrs_indicators, df_temp])

df_adrs_indicators.columns = df_adrs_indicators.columns.astype(str)
print("Data successfully imported.")
df_adrs_indicators

Data successfully imported.


,Recommend.Other,Recommend.All,Recommend.MA,RSI,RSI[1],Stoch.K,Stoch.D,Stoch.K[1],Stoch.D[1],CCI20,...,Pivot.M.Demark.R1,open,P.SAR,BB.lower,BB.upper,AO[2],volume,change,low,high
ADANIENT,-0.272727,-0.336364,-0.400000,42.908061,46.450466,38.571377,57.557260,62.927563,69.791253,-47.166799,...,3273.55005,3101.0000,3068.844290,3026.888675,3300.121335,19.055003,1134184,-1.304234,3100.00000,3149.20000
ADANIPORTS,-0.181818,-0.224242,-0.266667,46.121566,48.252895,35.109503,48.686492,50.601121,57.978247,-26.801626,...,1435.57500,1302.0000,1422.623000,1227.972527,1421.057473,37.260441,2696593,-0.851355,1294.40000,1322.00000
APOLLOHOSP,0.090909,0.178788,0.266667,50.521742,49.030196,53.187058,66.453874,65.224770,74.622892,-7.480179,...,6671.05005,6299.9502,6551.520000,6045.251709,6586.108251,132.749982,290190,0.501338,6202.00000,6347.60010
ASIANPAINT,0.000000,-0.466667,-0.933333,42.932325,44.953985,25.753136,39.102503,43.445044,49.831012,-66.970145,...,2934.39990,2827.6499,2940.632002,2798.676117,2911.293853,23.405007,911249,-0.506254,2820.00000,2841.45000
AXISBANK,0.000000,-0.333333,-0.666667,44.495294,46.532134,48.766328,65.777480,67.648803,79.224311,-29.591960,...,1089.57500,1049.9500,1096.400000,1026.225538,1085.329462,1.588382,7107881,-0.571861,1044.35000,1057.85000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OFSS,-0.363636,-0.315152,-0.266667,38.040142,42.554178,3.617302,11.321322,10.644158,21.272022,-166.253794,...,9479.77490,8013.0000,8861.856017,7929.923348,9100.246622,260.579332,200925,-2.471594,7840.70020,8081.35010
PVRINOX,0.000000,-0.066667,-0.133333,52.669405,55.631719,75.360023,84.016136,85.905724,88.922253,64.318177,...,1376.77500,1370.0000,1346.522578,1269.590505,1439.949495,29.455147,632602,-0.780830,1370.00000,1416.30000
RBLBANK,0.000000,-0.333333,-0.666667,43.696212,50.252723,47.850445,65.051570,67.737735,75.941390,-0.250875,...,258.77500,247.0000,266.750000,223.113496,266.941505,6.400293,7262079,-2.848602,242.10001,249.60001
NIFTY,-0.181818,-0.224242,-0.266667,45.031573,48.678732,46.361624,67.696975,70.945758,83.679810,-37.189081,...,22834.95050,22125.3010,22775.699000,21734.672188,22833.377612,361.558300,317267066,-0.559434,22079.44900,22213.75000


In [ ]:
volatility_threshold = 1
print(df_adrs_indicators.shape[0])
# rolling average of volume for the last 20 days
df_adrs_indicators['Volume_MA20'] = df_adrs_indicators['volume'].rolling(window=20).mean()

#rolling standard deviation of the logarithmic returns for the last 20 days
df_adrs_indicators['Volatility'] = df_adrs_indicators['close'].pct_change().rolling(window=20).std()

df_adrs_indicators['Bollinger_Band_Width'] = df_adrs_indicators['BB.upper'] - df_adrs_indicators['BB.lower']

df_adrs_indicators['Squeeze_Threshold'] = 0.04 * df_adrs_indicators['close']

# df_filtered = df_adrs_indicators[
#     (df_adrs_indicators['SMA5'] > df_adrs_indicators['SMA20']) &
#     (df_adrs_indicators['SMA20'] < df_adrs_indicators['SMA30'])
# ]

#  stocks where the price crosses above the 20-day SMA but remains below the 50-day and 200-day SMAs
# df_filtered = df_adrs_indicators[
#     (df_adrs_indicators['close'] > df_adrs_indicators['SMA20']) &
#     (df_adrs_indicators['close'] < df_adrs_indicators['SMA50']) &
#     (df_adrs_indicators['close'] < df_adrs_indicators['SMA200'])
# ]

df_filtered = df_adrs_indicators

df_filtered = df_filtered[df_filtered['Bollinger_Band_Width'] <  df_filtered['Squeeze_Threshold']]

#  stocks with EMA crossing SMA
# df_filtered = df_filtered[df_filtered['EMA50'] >= df_filtered['EMA200']]

# stocks with RSI above 60 (overbought)
# df_filtered = df_filtered[df_filtered['RSI'] < 50]

# stocks with positive MACD (signal line crossover)
# df_filtered = df_filtered[df_filtered['MACD.macd'] > df_filtered['MACD.signal']]

# # stocks where volume is bigger than the average of the last 20 days' volume
# df_filtered = df_adrs_indicators[df_adrs_indicators['volume'] > df_adrs_indicators['Volume_MA20']]

# df_filtered = df_filtered[df_filtered['volume'] > df_filtered['Volume_MA20']]

#  stocks with ADX above a certain threshold (indicating a strong trend)
# df_filtered = df_filtered[df_filtered['ADX'] > 30]

# # stocks with BBPOWER and Rec.BBPower conditions Bulls and bear Power indicator
# df_filtered = df_filtered[(df_filtered['BBPower'] > 1) & (df_filtered['Rec.BBPower'] != 'SELL')]

# # #stocks with VWMA and Rec.VWMA conditions
# df_filtered = df_filtered[(df_filtered['VWMA'] < df_filtered['close']) & (df_filtered['Rec.VWMA'] != 'SELL')]

# #volatility
# df_filtered = df_filtered[df_filtered['Volatility'] > volatility_threshold]

# Print the stocks that meet the criteria
filtered_shares = df_filtered.index.to_list()

print('Shares filtered:', list(set(filtered_shares)))
print(len(list(set(filtered_shares))))


1183
Shares filtered: ['ASIANPAINT', 'ICICIBANK', 'BHARTIARTL', 'ERIS']
4


In [ ]:
# Index(['Recommend.Other', 'Recommend.All', 'Recommend.MA', 'RSI', 'RSI[1]',
#        'Stoch.K', 'Stoch.D', 'Stoch.K[1]', 'Stoch.D[1]', 'CCI20', 'CCI20[1]',
#        'ADX', 'ADX+DI', 'ADX-DI', 'ADX+DI[1]', 'ADX-DI[1]', 'AO', 'AO[1]',
#        'Mom', 'Mom[1]', 'MACD.macd', 'MACD.signal', 'Rec.Stoch.RSI',
#        'Stoch.RSI.K', 'Rec.WR', 'W.R', 'Rec.BBPower', 'BBPower', 'Rec.UO',
#        'UO', 'close', 'EMA5', 'SMA5', 'EMA10', 'SMA10', 'EMA20', 'SMA20',
#        'EMA30', 'SMA30', 'EMA50', 'SMA50', 'EMA100', 'SMA100', 'EMA200',
#        'SMA200', 'Rec.Ichimoku', 'Ichimoku.BLine', 'Rec.VWMA', 'VWMA',
#        'Rec.HullMA9', 'HullMA9', 'Pivot.M.Classic.S3', 'Pivot.M.Classic.S2',
#        'Pivot.M.Classic.S1', 'Pivot.M.Classic.Middle', 'Pivot.M.Classic.R1',
#        'Pivot.M.Classic.R2', 'Pivot.M.Classic.R3', 'Pivot.M.Fibonacci.S3',
#        'Pivot.M.Fibonacci.S2', 'Pivot.M.Fibonacci.S1',
#        'Pivot.M.Fibonacci.Middle', 'Pivot.M.Fibonacci.R1',
#        'Pivot.M.Fibonacci.R2', 'Pivot.M.Fibonacci.R3', 'Pivot.M.Camarilla.S3',
#        'Pivot.M.Camarilla.S2', 'Pivot.M.Camarilla.S1',
#        'Pivot.M.Camarilla.Middle', 'Pivot.M.Camarilla.R1',
#        'Pivot.M.Camarilla.R2', 'Pivot.M.Camarilla.R3', 'Pivot.M.Woodie.S3',
#        'Pivot.M.Woodie.S2', 'Pivot.M.Woodie.S1', 'Pivot.M.Woodie.Middle',
#        'Pivot.M.Woodie.R1', 'Pivot.M.Woodie.R2', 'Pivot.M.Woodie.R3',
#        'Pivot.M.Demark.S1', 'Pivot.M.Demark.Middle', 'Pivot.M.Demark.R1',
#        'open', 'P.SAR', 'BB.lower', 'BB.upper', 'AO[2]', 'volume', 'change',
#        'low', 'high'],
#       dtype='object')